## This code is a BolingerBandsBacktest using the historical data from Finalterm.

# Import

In [ ]:
!pip install python-binance # This is an unofficial Python wrapper for the Binance exchange REST API v3.
import pandas as pd # Python Data Analysis Library
from binance.client import Client # Creating a client
from binance_keys import api_key, secret_key # Issued from a personal account to use binance data
import time # Time data
from datetime import datetime # Time data
import plotly.graph_objects as go
from plot_utils import plot_results
client = Client(api_key, secret_key)

# GatherData

In [ ]:
symbols = ['BTC','ETH','LTC'] # Symbol string 

# Data collection
def gather_data(symbols, start_n_hours_ago): 
    merge = False
    for symbol in symbols: 
      # Get Historical Klines from Binance, symbol: Name of symbol pair, interval : Binance Kline interval, start_str : optional - Start date string in UTC format or timestamp in milliseconds
      # Returns : list of OHLCV values (Open time, Open, High, Low, Close, Volume, Close time, Quote asset volume, Number of trades, Taker buy base asset volume, Taker buy quote asset volume, Ignore)
        klines = client.get_historical_klines(symbol=f'{symbol}USDT',
                                              interval=client.KLINE_INTERVAL_1HOUR, 
                                              start_str=str(datetime.now()-timedelta(hours=start_n_hours_ago)))
        cols = ['OpenTime',
                f'{symbol}-USD_Open',
                f'{symbol}-USD_High',
                f'{symbol}-USD_Low',
                f'{symbol}-USD_Close',
                f'{symbol}-USD_volume', #
                'CloseTime',
                f'{symbol}-QuoteAssetVolume',
                f'{symbol}-NumberOFTrades',
                f'{symbol}-TBBAV',
                f'{symbol}-TBQAV',
                f'{symbol}-ignore']

        df = pd.DataFrame(klines,columns=cols)

        if merge == True:
            dfs = pd.merge(df,dfs,how='inner',on=['OpenTime','CloseTime'])
        else :
            dfs = df
            merge = True

    dfs['OpenTime'] = [datetime.fromtimestamp(ts/1000) for ts in dfs['OpenTime']]
    dfs['CloseTime'] = [datetime.fromtimestamp(ts/1000) for ts in dfs['CloseTime']]     
   
    for col in dfs.columns :
        if not 'Time' in col:
            dfs[col] = dfs[col].astype(float)

fig = go.Figure(data=[go.Candlestick(x=dfs['OpenTime'],
                open=dfs['BTC-USD_Open'],
                high=dfs['BTC-USD_High'],
                low=dfs['BTC-USD_Low'],
                close=dfs['BTC-USD_Close'])])

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

dfs.to_csv('BTC_ETH_LTC_Jan2721_Jul2121_1h.csv',index=False)

# BollingerBandsBacktest

In [ ]:
df = pd.read_csv('BTC_ETH_LTC_Jan2721_Jul2121_1h.csv')

# Boliger band function
def sma(data, window): # Moving average
    return(data.rolling(window = window).mean())  # Average the closing price during the window

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std() # Number of standard deviation (typically 2)
    upper_band = sma + std * nstd # nstd is Standard deviation over last n periods 
    lower_band = sma - std * nstd
    
    return upper_band, lower_band

symbols = ['BTC','ETH','LTC']

nstd = 3

for symbol in symbols:
    df[f'{symbol}_sma'] = sma(df[f'{symbol}-USD_Open'], 20)
    df[f'{symbol}_upper_band'], df[f'{symbol}_lower_band'] = bollinger_band(df[f'{symbol}-USD_Open'], df[f'{symbol}_sma'], 20, nstd)
    
df.dropna(inplace=True)

class TradingEnv:
    def __init__(self, balance_amount, balance_unit, trading_fee_multiplier):
        self.balance_amount = balance_amount # the number of currencies held
        self.balance_unit = balance_unit # Currency
        self.buys = []
        self.sells = []
        self.trading_fee_multiplier = trading_fee_multiplier # Transaction fee
        
    def buy(self, symbol, buy_price, time):
        self.balance_amount = (self.balance_amount / buy_price) * self.trading_fee_multiplier
        self.balance_unit = symbol
        self.buys.append([symbol, time, buy_price])
        
    def sell(self, sell_price, time):
        self.balance_amount = self.balance_amount * sell_price * self.trading_fee_multiplier
        self.sells.append( [self.balance_unit, time, sell_price] )
        self.balance_unit = 'USDT'

# VIP level 0, paying fees with BNB = 0.075%
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925)

for i in range(len(df)):
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]: #buy signal
                env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                break
                
    if env.balance_unit != 'USDT':
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]: #sell signal
            env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])

print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

plot_results(df, 'LTC', env.buys, env.sells)